<a href="https://colab.research.google.com/github/daizybhadresha/dsprojects/blob/main/MCFS_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## Loading the data

import pandas as pd

path1 = '/content/drive/My Drive/Capstone/capstone_data/Customer_Data.xlsx'
path2 = '/content/drive/My Drive/Capstone/capstone_data/Plant Master.xlsx'
path3 = '/content/drive/My Drive/Capstone/capstone_data/Invoice_Preprocesed.csv'
path4 = '/content/drive/My Drive/Capstone/capstone_data/JTD.csv'

cust_df = pd.read_excel(path1)
plant_df = pd.read_excel(path2)
invoice_df = pd.read_csv(path3)
jtd_df = pd.read_csv(path4)

print(cust_df.columns)
print(plant_df.columns)
print(invoice_df.columns)
print(jtd_df.columns)

## Cleaning District
#mergeWith(invoice_df[], plant_df, '', '', '')


Index(['Business Partner', 'Customer No.', 'Partner Type', 'Data Origin',
       'Title', 'Marital Status', 'Occupation', 'Date of Birth', 'Death date'],
      dtype='object')
Index(['Plant', 'Name 1', 'Valuation Area', 'Customer no. - plant',
       'Vendor number plant', 'Factory calendar', 'Name 2',
       'House number and street', 'PO Box', 'Postal Code', 'City',
       'Sales organization', 'State'],
      dtype='object')
Index(['Unnamed: 0', 'Area / Locality', 'CITY', 'Cust Type', 'Customer No.',
       'District', 'Invoice Date', 'Invoice No', 'Invoice Time', 'Job Card No',
       'JobCard Date', 'JobCard Time', 'KMs Reading', 'Labour Total', 'Make',
       'Misc Total', 'Model', 'OSL Total', 'Order Type', 'Parts Total',
       'Pin code', 'Plant', 'Plant Name1', 'Technician Name',
       'Total Amt Wtd Tax.'],
      dtype='object')
Index(['Unnamed: 0', 'DBM Order', 'Order Item', 'Material',
       'Labor Value Number', 'Description', 'Item Category', 'Order Quantity',
       '

EDA Questions solved in the below EDAs

**Customer Analysis (segmentation) :**

Customer type I have dealt with and the most

From customer types ..frequency of each coming in & revenue it generated

Which customer I should give priority to ?
Is he/she might new/regular customer? How much has he/she invested till now?
Depending on the service & make/model identify how much avg labour & how much avg parts costs will be required.

**Service Analysis :**

Service Type - Which service type (Running repair, accidental, warranty) generates how much  money?

Type of order each state receives

What kind of service is required by each car

**Plant Analysis**

Which car comes/visits which plant/area   Ex Repair for Ford cars comes alot, areawise may be

How profitable the garage is ? Frequency of Customers/Orders ? Total Revenue ?


**Make/Model Analysis :**

For each model, how many count of orders & how much revenue they generate?  (Which model will help me generate more money)

Avg km reading at which the servicing required?  (Overall/ Car / Make/ Model wise)
With ewf kms reading - these services are used ...etc etc







**Analyzing customer value: Recency, Frequency, and Monetary Value (RFM)**

**Recency:** Customers who have made a purchase within the last

four months are given - 3.
four to eight months are given - 2. 
more than eight - 1.

**Frequency**: Avg. frequency in an year right from 2012-2016

Once visited user - 1
More than once to the avg vits by the user - 2
More than the avg visits - 3

**Monetary:** Avg. Transaction cost X Avg. Frequency



In [ ]:

import datetime

def findRecency(recent_date):
  # print(max(cust_recency['Invoice Date']))
  # Since data is very old(2012-2016), taking the latest date in the dataframe as today to get appropriate data
  today = datetime.datetime.strptime("2016-10-31", '%Y-%m-%d').date()    # datetime.date.today()
  margin = datetime.timedelta(days=30*4)
  date_in_last_4_months = today - margin <= recent_date <= today
  date_in_last_4_to_8_months = today - datetime.timedelta(days=30*8) <= recent_date <=  today - datetime.timedelta(days=30*4+1)
  date_in_last_8_to_12_months = today - datetime.timedelta(days=30*12) <= recent_date <=  today - datetime.timedelta(days=30*8+1)
  recency = 3 if date_in_last_4_months else 2 if date_in_last_4_to_8_months else 1
  return recency
  
def findFrequency():
  invoice_df['year'] = invoice_df['Invoice Date'].apply(lambda x: x[0:4])
  cust_freq = invoice_df[['Customer No.', 'year']].groupby(['Customer No.','year'])['year'].agg({"count_yearwise": "count"})
  cust_freq = cust_freq.groupby(['Customer No.'])['count_yearwise'].agg({"avg_frequency": "mean"})
  avg_visits = cust_freq['avg_frequency'].mean()
  cust_freq['frequency'] = cust_freq['avg_frequency'].apply(lambda x: 1 if x==1 else 2 if (x>1 and x<=avg_visits) else 3)
  return cust_freq

def findMonetaryValue():
  # Avg. transaction cost
  cust_monval = invoice_df[['Customer No.', 'Invoice Date', 'Total Amt Wtd Tax.']].groupby(['Customer No.'])['Total Amt Wtd Tax.'].agg({"avg_transaction_cost": "mean"})
  cust_freq['avg_frequency']
  cust_monval = pd.merge(cust_monval, cust_freq[['avg_frequency']], right_on=cust_freq.index, left_on=cust_monval.index, how='inner')
  cust_monval['monetary_value'] = cust_monval['avg_transaction_cost'] * cust_monval['avg_frequency']	
  cust_monval.rename(columns = {"key_0": "Customer No."}, inplace = True)
  avg_trans_cost = cust_monval['monetary_value'].mean()
  cust_monval['monetary_val'] = cust_monval['monetary_value'].apply(lambda x: 1 if x==1 else 2 if (x>1 and x<=avg_trans_cost) else 3)
  return cust_monval

cust_value = invoice_df[['Customer No.', 'Invoice Date']].groupby(['Customer No.'])
cust_recency = cust_value.apply(lambda x: x.sort_values(['Invoice Date'], ascending = False).head(1)).reset_index(drop=True)


cust_recency['recency'] = cust_recency['Invoice Date'].apply(lambda x: findRecency(datetime.datetime.strptime(x, '%Y-%m-%d').date() ))
cust_recency

cust_freq = findFrequency()
cust_freq

cust_monval = findMonetaryValue()
cust_monval

# Merging all the 3 values with customer no in a single dataframe
cust_lifetime_value = cust_recency.merge(cust_freq,on='Customer No.').merge(cust_monval,on='Customer No.')[['Customer No.', 'recency', 'frequency', 'monetary_val']]
print(cust_lifetime_value['recency'].value_counts())
print(cust_lifetime_value['frequency'].value_counts())
print(cust_lifetime_value['monetary_val'].value_counts())
cust_lifetime_value

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)



,Customer No.,avg_transaction_cost,avg_frequency,monetary_value,monetary_val
0,1,0.000000,2.000000,0.000000,3
1,10,2221.112000,2.500000,5552.780000,2
2,100,10399.870000,2.000000,20799.740000,3
3,1000,1468.906667,2.000000,2937.813333,2
4,10000,247.931429,2.333333,578.506667,2
...,...,...,...,...,...
253479,PLX241,3477.370000,1.000000,3477.370000,2
253480,PLX277,6870.950000,1.000000,6870.950000,3
253481,PLX312,567.800000,1.000000,567.800000,2
253482,PLX380,4346.980000,1.000000,4346.980000,2


In [ ]:
### Segmenting customer based on RFM Score
cust_lifetime_value['RFM Score'] =  cust_lifetime_value['recency']	+ cust_lifetime_value['frequency'] + cust_lifetime_value['monetary_val']	
cust_lifetime_value['RFM Score'].value_counts()

4    74525
6    63419
5    47102
7    37565
8    18786
9    12086
3        1
Name: RFM Score, dtype: int64

In [ ]:
print("Most Valuable Customers")
print("----------------------------")
print(cust_lifetime_value[cust_lifetime_value['RFM Score']>=8])
print("Somewhat Valuable Customers")
print("----------------------------")
print(cust_lifetime_value[(cust_lifetime_value['RFM Score']>=6) & (cust_lifetime_value['RFM Score']<=7)])
print("Least Valuable Customers")
print("----------------------------")
print(cust_lifetime_value[(cust_lifetime_value['RFM Score']>=3) & (cust_lifetime_value['RFM Score']<=5)])

Most Valuable Customers
----------------------------
       Customer No.  recency  frequency  monetary_val  RFM Score
12           100007        3          3             3          9
24           100018        3          3             3          9
29           100022        3          3             3          9
42           100035        3          3             2          8
51           100043        3          3             2          8
...             ...      ...        ...           ...        ...
253472      MFCWP01        2          3             3          8
253473        MFM15        3          3             3          9
253475       PLX076        3          3             3          9
253477       PLX099        3          3             2          8
253478       PLX109        3          3             3          9

[30872 rows x 5 columns]
Somewhat Valuable Customers
----------------------------
       Customer No.  recency  frequency  monetary_val  RFM Score
0                 1

In [ ]:
# ? Customer type I have dealt with and the most
invoice_df['Cust Type'].value_counts()


Retail              488995
Fleets                2218
Corporate others       579
Corporate- M&M         254
MFCWL                  252
Franchise               15
Insurance                1
Name: Cust Type, dtype: int64

In [ ]:
# ? Which services are received the most
invoice_df['Order Type'].value_counts()

Running Repairs      269455
Paid Service         135004
Accidental            44159
SMC Redemption        15738
Mechanical            15100
SMC Value Package      9781
Repeat Order           3043
WBW Order                17
Workshop Damage          17
Name: Order Type, dtype: int64

# **Service Order Analysis :**
 Which plants/districts/cieties has got how many orders 

Why?   1. #Garages  2.#Customers(Population)  3.People owing more cars(Person:Car ratio) 

In [ ]:
# ? Which plants/districts/cieties has got how many orders 

invoice_df['Make Model'] = invoice_df['Make'] + invoice_df['Model']
people_car = invoice_df.groupby(['Customer No.','Make Model'])['Make Model'].nunique()
print("Each customer owns only 1 car")
print(people_car.value_counts())

## Total Orders	# Population(Customers visisted)	# Garages
dist_df = invoice_df.groupby(['District'])
no_of_cust = dist_df['Customer No.'].nunique()
no_of_garages = dist_df['Plant'].nunique()

loc = invoice_df.groupby(['District'])['Plant'].count().sort_values(ascending=False, inplace=False).to_frame()
loc = loc.merge(no_of_cust, on='District').merge(no_of_garages, on='District').rename(columns={'Plant_x':'Total Orders', 'Customer No.':'# Population(Customers visisted)', 'Plant_y':'# Garages'})
loc

,Total Orders,# Population(Customers visisted),# Garages
District,,,
Maharashtra,128119,60409,141
Tamil Nadu,116309,57661,105
Karnataka,44135,24592,113
Andhra Pradesh,40604,19751,94
Uttar Pradesh,31534,16316,96
Gujarat,24403,12391,79
Punjab,22741,10871,56
Haryana,17971,8604,95
Madhya Pradesh,13598,9138,69


In [ ]:
## Make Model Analysis

print("\nWhat cars has come to which plant with frequency ?")
print("---------------------------------------------------")
make_mod_df = invoice_df.groupby(['Plant Name1','Make','Model'])['Make'].count()   #.sort_values(ascending=False, inplace=False)
print(make_mod_df)

## For each model - Avg service type price 

# Why?  1.Are their parts pricey   2.More Labour value is required    3.More service frequency

print("\nAverage cost of repair - Make wise based on service type?")
# print("----------------------------------------------------------")
# model_service_price = invoice_df.groupby(['Make', 'Order Type'])['Total Amt Wtd Tax.'].mean()#.sort_values(ascending=False, inplace=False)
# model_service_price

# Parts total - Make Type - Service type



What cars has come to which plant with frequency ?
---------------------------------------------------
Plant Name1              Make        Model   
AMBATTUR - CHENNAI       AUDI        A4            2
                                     Q5            1
                         BMW         5 SERIES      1
                                     X3            1
                         DAEWOO      MATIZ         1
                                                ... 
YESHWANTPUR - BANGALORE  TOYOTA      INNOVA      175
                                     QUALIS       31
                         VOLKSWAGEN  JETTA         5
                                     POLO         55
                                     VENTO        34
Name: Make, Length: 19166, dtype: int64


# What kind of service is required by each car. How often - Revenue it can generate

In [ ]:
## ? What kind of service is required by each car and how often - how much revenue it can generate

## Average cost = Total Cost/ Total orders
print("\nAverage cost required by each model for each service ?")  
print("--------------------------------------------------------------")
make_mod_df_2 = invoice_df.groupby(['Make', 'Model', 'Order Type'])['Make', 'Model', 'Total Amt Wtd Tax.'].agg({"Make": {"Total Orders": "count"},"Total Amt Wtd Tax.": {"Total Cost": "sum"}})
make_mod_df_2['Average Cost'] = make_mod_df_2['Total Cost'] / make_mod_df_2['Total Orders']
make_mod_df_3 = make_mod_df_2[['Average Cost']]
make_mod_df_3 = make_mod_df_3.reset_index()
make_mod_df_3['Make Model'] =  make_mod_df_3['Make'] + " " + make_mod_df_3['Model']
print(make_mod_df_3)

import plotly.express as px
#data_canada = px.data.gapminder().query("country == 'Canada'")
fig = px.bar(make_mod_df_3,  x="Make Model" ,y='Average Cost',  hover_data=['Order Type'])
fig.show()



Average cost required by each model for each service ?
--------------------------------------------------------------
       Make       Model       Order Type  Average Cost        Make Model
0      AUDI          A4       Accidental  18980.842222           AUDI A4
1      AUDI          A4       Mechanical   2599.740000           AUDI A4
2      AUDI          A4     Paid Service   2640.450000           AUDI A4
3      AUDI          A4  Running Repairs   4146.273415           AUDI A4
4      AUDI          A6       Accidental  29156.341111           AUDI A6
...     ...         ...              ...           ...               ...
1186  VOLVO   VOLVO S80     Paid Service   1883.845000   VOLVO VOLVO S80
1187  VOLVO   VOLVO S80  Running Repairs    394.375000   VOLVO VOLVO S80
1188  VOLVO  VOLVO XC90       Accidental   8701.570000  VOLVO VOLVO XC90
1189  VOLVO  VOLVO XC90     Paid Service   1480.000000  VOLVO VOLVO XC90
1190  VOLVO  VOLVO XC90  Running Repairs   3712.737000  VOLVO VOLVO XC90

[119

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1455: FutureWarning:

using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))




# Which order type (Running repair, accidental, warranty) generates how much money?

In [ ]:


## How much avg. money each service has generated

services_list = invoice_df['Order Type'].unique()
print("Services List : " , services_list)

print("\nAverge Revenue / Service")
print("-----------------------------------------")
services_df_1 = invoice_df.groupby(['Order Type'])['Total Amt Wtd Tax.'].agg(['mean', 'count', 'sum']).rename(columns={'mean':'Average', 'count':'Total Orders', 'sum':'Total Revenue'}).sort_values(ascending=False, inplace=False, by='Average')
print(services_df_1)

## General Trend of Each Services - Frequency

serv_df_1 = invoice_df.groupby(['JobCard Date', 'Order Type'])['CITY'].count().to_frame().unstack()
serv_df_1 = serv_df_1.reset_index().rename(columns={'CITY':'# Orders'})
dates = invoice_df['Invoice Date'].to_list()

import plotly.graph_objects as go
fig = go.Figure()
print("\n\nNo. of Orders from " +min(dates)+" to "+max(dates))
print("-----------------------------------------")

for service in services_list:
  fig.add_trace(go.Scatter(x=serv_df_1['JobCard Date'], y=serv_df_1['# Orders',service], name=service))
fig.update_layout(xaxis_rangeslider_visible=True)  # title_text= , 
fig.show()


Services List :  ['Paid Service' 'SMC Value Package' 'Running Repairs' 'SMC Redemption'
 'Accidental' 'Repeat Order' 'Workshop Damage' 'Mechanical' 'WBW Order']

Averge Revenue / Service
-----------------------------------------
                        Average  Total Orders  Total Revenue
Order Type                                                  
Accidental         17107.840265         44159   7.554651e+08
WBW Order           7722.238235            17   1.312780e+05
Paid Service        4778.902440        135004   6.451709e+08
SMC Value Package   3586.512683          9781   3.507968e+07
Mechanical          3142.749370         15100   4.745552e+07
Running Repairs     2518.905208        269455   6.787316e+08
SMC Redemption       106.381010         15738   1.674224e+06
Repeat Order           1.316707          3043   4.006740e+03
Workshop Damage        0.000000            17   0.000000e+00


No. of Orders from 2012-04-02 to 2016-10-31
-----------------------------------------


# Order Types received District wise

Spending pattern (services could be suited to the particular spending pattern)

In [ ]:
# Plotting of order types according to the state - plotly
 import plotly.graph_objects as go

table = invoice_df.groupby(['District', 'Order Type'])['Order Type'].count().unstack() 
types = table.index.values

clist = [round(float(np_float),2) for np_float in table['Accidental'].values]

fig = go.Figure(go.Bar(x=types, y=clist, name='Accidental'))

for col in table.columns[1:]:
  #print(type(types),type(table[col].values))
  clist = [round(float(np_float),2) for np_float in table[col].values]
  fig.add_trace(go.Bar(name=col, x=types, y=clist))

# Change the bar mode
fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})
fig.show()

In [ ]:
## Get total parts involved in each repair/make-model

def returnAll(df, col, val):
  return df[df[col]==val]

def mergeWith(df1, df2, right, left, how):
  return pd.merge(df1, df2, right_on=right, left_on=left, how=how)

parts_per_job = jtd_df[].groupby(['DBM Order'])['Order Item'].count().reset_index()
#print(parts_per_job[parts_per_job['DBM Order']==168303])
df = mergeWith(invoice_df, parts_per_job, 'DBM Order', 'Job Card No', 'left')
parts_per_job_make = df[['Job Card No', 'Order Type', 'Make', 'Model', 'Order Item']]
parts_per_job_make

print(invoice_df['Job Card No'].nunique(),  parts_per_job_make['Job Card No'].nunique())
#print(len(invoice_df[invoice_df['Parts Total']!=0]))
returnAll(jtd_df, 'DBM Order', 173997)
#print(jtd_df[jtd_df['DBM Order']==173997])  # 173997
returnAll(invoice_df, 'Job Card No', 173997)

def getDuplicates(df, cols):
  return df[df.duplicated(subset=cols, keep=False)]

def groupBY(df, cols):
  return df.groupby(cols)

# ['Unnamed: 0', 'Area / Locality', 'CITY', 'Cust Type', 'Customer No.', 'District', 'Invoice Date', 'Invoice No', 'Invoice Time', 'Job Card No', 'JobCard Date', 'JobCard Time', 'KMs Reading', 'Labour Total', 'Make',
#  'Misc Total', 'Model', 'OSL Total', 'Order Type', 'Parts Total', 'Pin code', 'Plant', 'Plant Name1', 'Technician Name','Total Amt Wtd Tax.']

groupBY(invoice_df,['Customer No.', 'Job Card No', 'Order Type', 'Pin code', 'Technician Name'])

# GroupBy - Job card no. - Adding each Parts Total, OSL Total, Misc Total, Labour Total      # Total Amt = Parts Total + OSL Total + Misc Total + Labour Total
trimmed_invoice = groupBY(invoice_df, ['Job Card No'])[['Parts Total', 'OSL Total', 'Misc Total', 'Labour Total', 'Total Amt Wtd Tax.']].sum()
trimmed_invoice[trimmed_invoice.index ==177979]


489329


,Unnamed: 0,DBM Order,Order Item,Material,Labor Value Number,Description,Item Category,Order Quantity,Target quantity UoM,Net value
4248244,4248244,173997,10,TMIC_VAS8,TMICIP0030,Interior & Upholstery Cleaning,P001,1.0,MIN,392.81
4248245,4248245,173997,20,TATAMOTORS_SRV,TMICPS0004,WASHING,P001,84.0,MIN,74.79
4248246,4248246,173997,30,TATAMOTORS_SRV,TMICPS0004,WASHING,P001,84.0,MIN,74.79
4248247,4248247,173997,40,TMIC_MENU3,TMICWH0013,Wheel Alignment,P001,1.0,MIN,261.87
4248248,4248248,173997,50,MISC_SRV13,NaN,MISC_ON ROAD ASSISTANCE WITHOUT TOWING,P011,1.0,MIN,197.03
